In [1]:
import glob
import os

# import matplotlib as mpl
import matplotlib.pyplot as plt
import palettable.colorbrewer.diverging as pld

import numpy as np
import pandas as pd

# from sklearn import preprocessing
# from scipy import integrate  
# import scipy.signal as scipy
# from scipy.optimize import curve_fit

%matplotlib widget


In [2]:
## Plotting parameters

palette = pld.RdYlGn_4_r
cmap = palette.mpl_colormap
transparency = 0.7
labelsize = 22
spinewidth = 2
fontsize = 26
legendsize = 13
fontweight = 'bold'
labelpad = 6
pad = 10
linewidth = 2.2
markersize = 6


In [ ]:
## In the current working folder (cwd)
## Find the datafiles of .cal for best fitting
# cwd = os.getcwd()
cwd = '/Users/chenghunglin/Desktop/test/20230608_CsPbBr_PDF_PAR_30nm/DFA_PDF_06-6'
cal_files = glob.glob(cwd + '/**.cal')
cal_files.sort()
print(*cal_files, sep = '\n')  ## Print one element per line


## Find the datafiles of 2D size distribution
mtx_files = glob.glob(cwd + '/**.mtx')
mtx_files.sort()
print(*mtx_files, sep = '\n')  ## Print one element per line


In [ ]:
## Read **_Best.cal into pandas dataframe
df0 = pd.read_csv(cal_files[0], header=None, skiprows=2, sep=' ')
df = df0.dropna(axis=1).rename(columns={0:'theta', 2:'obs', 8:'cal', 14:'cal1', 20:'cal2', 26:'bkg'})


## Read **_plot2D.mtx into pandas dataframe
df1 = pd.read_csv(mtx_files[0], skiprows=1, sep=' ', skipinitialspace=True)
updated_labels = df1.columns.values.tolist()
del updated_labels[0]
updated_labels.append('None')
df1.columns = updated_labels

In [ ]:
## Export DSE calculated data and bkg
fn_cal = cwd + '/CsPbBr3_20211114-120845_DSE_0608_06-6.xy'
fn_bkg = cwd + '/Tol_Ol_Solvent_20211113-184203_DSE_0608_06-6.xy'

df_cal = pd.DataFrame()
df_cal['tth'] = df['theta']
df_cal['Int_cal'] = df['cal']

df_bkg = pd.DataFrame()
df_bkg['tth'] = df['theta']
df_bkg['Int_bkg'] = df['bkg']

# df_cal.to_csv(fn_cal, sep=' ', header=False, index=False, float_format='{:.8e}'.format)
# df_bkg.to_csv(fn_bkg, sep=' ', header=False, index=False, float_format='{:.8e}'.format)

In [ ]:
## Plot **_Best.cal
f1, ax1 = plt.subplots(1, 1, figsize = (10, 6))
data_range = 1200 #648 # < 12 degree
ax1.plot(df.theta[:data_range], df.obs[:data_range], label='Observation', marker='o',color='dimgrey', fillstyle='none', linestyle='None', markersize=markersize)
ax1.plot(df.theta[:data_range], df.cal[:data_range], 'r-', label='DSE Simulation', linewidth=linewidth)
ax1.plot(df.theta[:data_range], df.bkg[:data_range], 'g-', label='Background', linewidth=linewidth)

y_offset = 1500
difference = df.obs - df.cal + y_offset
ax1.plot(df.theta[:data_range], difference[:data_range], 'b-', label='Difference', linewidth=linewidth)
ax1.hlines(y=0+y_offset, xmin=df.theta[0], xmax=df.theta[data_range], color='k', linestyle='-')

legend_properties = {'size':legendsize, 'weight':'bold'}
# ax1.legend(prop=legend_properties, bbox_to_anchor=(0.31, 0.30))

# ax1.set_xticks([])
ax1.tick_params(axis='x', direction='in', labelsize=labelsize)
#plt.setp(ax1.get_xticklabels(), fontweight="bold")
ax1.spines["bottom"].set_linewidth(spinewidth)
ax1.set_yticks([])
ax1.spines["left"].set_linewidth(spinewidth)
ax1.spines["top"].set_linewidth(spinewidth)
ax1.spines["right"].set_linewidth(spinewidth)
#ax1.spines['left'].set_visible(False)

imag_name = 'Best_cal_06-6.png'
# plt.savefig(imag_name, dpi = 600,  transparent=True)

'''
## Plot **.mtx as an inset 2D plot in ax1
from mpl_toolkits.axes_grid.inset_locator import inset_axes, InsetPosition
## Method 1:
ax2 = plt.axes(projection='3d')
#ax2 = plt.axes([0,0,1,1])
ip = InsetPosition(ax1, [0.45, 0.45, 0.45, 0.45])
ax2.set_axes_locator(ip)
## Method 2:
ax2 = inset_axes(ax1, width="120%", height="120%",
                   bbox_to_anchor=(0.45, 0.45, 0.45, 0.45),
                   bbox_transform=ax1.transAxes, loc=10)
'''

In [ ]:
## Plot **.mtx as an 3D plot in ax2
f2 = plt.figure(figsize = (10, 6))
ax2 = plt.axes(projection='3d')
# df_reshape = df1.pivot(index='n_ab', columns='n_c', values='Number_Frac.')
df_reshape = df1.pivot(index='Base_Diam.(e.c.)[nm]', columns='Height[nm]', values='Number_Frac.')

X = np.array(df_reshape.index)
Y = np.array(df_reshape.columns)
X, Y = np.meshgrid(Y, X)
Z = np.array(df_reshape)
Z[Z<0.001] = np.nan

surf = ax2.plot_surface(X, Y, Z, vmin=0, vmax=0.115, rstride=1, cstride=1, cmap=cmap)
cbar = f2.colorbar(surf, shrink=0.5, aspect=10)

ax2.set_xlim(5, 13)   ## L_c
ax2.set_ylim(5, 13)   ## D_ab
ax2.view_init(20, 70)


#ax2.set_xticks([])
ax2.tick_params(axis='x', direction='in', labelsize=labelsize-4)

# ax2.set_yticks([])
ax2.set_yticks(np.arange(ax2.get_ylim()[0], ax2.get_ylim()[1], 2))
ax2.tick_params(axis='y', direction='in', labelsize=labelsize-4)

z_step = 0.05; z0 = 0; z1 = np.nanmax(Z)+z_step*0
ax2.set_zticks(np.arange(z0, z1, z_step))
ax2.tick_params(axis='z', direction='in', labelsize=labelsize-4, pad=10)

cbar.set_ticks(np.arange(z0, z1, z_step))
cbar.ax.tick_params(labelsize=labelsize-4)

imag_name = 'Size_dis_3D_06-6.png'
# plt.savefig(imag_name, dpi = 600,  transparent=True)

In [ ]:
np.nanmax(Z)

In [ ]:
df_reshape